In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import AUC, Precision, Recall
from sklearn.utils.class_weight import compute_class_weight

# 📁 Paths
BASE_DIR = "C:\\Users\\MANJU\\Desktop\\FYP_Diabetes\\data"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "val")
TEST_DIR = os.path.join(BASE_DIR, "test")

# 🔢 Parameters
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 64
EPOCHS = 25

# 🧪 Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=45,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.3,
    shear_range=0.3,
    brightness_range=[0.6, 1.4],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# 📦 Data Loaders
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_generator = val_test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# 📏 Class Weight Calculation
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# 🧠 Load MobileNetV2 Base
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
base_model.trainable = False  # Initially freeze base model

# 🏗️ Custom Classifier Head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# 🧪 Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.0003),
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(name='auc'), Precision(name='precision'), Recall(name='recall')]
)

# 📌 Callbacks
checkpoint = ModelCheckpoint("best_mobilenetv2_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# 🚀 Phase 1: Train Custom Head
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    class_weight=class_weights,
    callbacks=[checkpoint, early_stop, lr_scheduler]
)

# 🔓 Phase 2: Unfreeze and Fine-tune MobileNetV2
base_model.trainable = True

# Re-compile with lower LR
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(name='auc'), Precision(name='precision'), Recall(name='recall')]
)

# Fine-tuning Training
history_finetune = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    class_weight=class_weights,
    callbacks=[checkpoint, early_stop, lr_scheduler]
)

# 🧾 Evaluate Model
loss, accuracy, auc, precision, recall = model.evaluate(test_generator, steps=len(test_generator))
print(f"\n✅ Final Test Accuracy: {accuracy * 100:.2f}%")
print(f"📈 AUC: {auc:.4f} | 🎯 Precision: {precision:.4f} | 🔍 Recall: {recall:.4f}")

# 💾 Save Final Model
model.save("final_mobilenetv2_model.h5")
print("✅ Optimized MobileNetV2 training complete!")


Found 112 images belonging to 2 classes.
Found 24 images belonging to 2 classes.
Found 24 images belonging to 2 classes.


C:\Users\MANJU\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5792 - auc: 0.5587 - loss: 0.8246 - precision: 0.6063 - recall: 0.6233 
Epoch 1: val_accuracy improved from -inf to 0.66667, saving model to best_mobilenetv2_model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 15s 5s/step - accuracy: 0.5796 - auc: 0.5608 - loss: 0.8236 - precision: 0.5955 - recall: 0.6239 - val_accuracy: 0.6667 - val_auc: 0.7500 - val_loss: 0.6010 - val_precision: 0.6429 - val_recall: 0.7500 - learning_rate: 3.0000e-04
Epoch 2/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4866 - auc: 0.5459 - loss: 0.8156 - precision: 0.5508 - recall: 0.4655
Epoch 2: val_accuracy improved from 0.66667 to 0.79167, saving model to best_mobilenetv2_model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step - accuracy: 0.5030 - auc: 0.5689 - loss: 0.7952 - precision: 0.5478 - recall: 0.4651 - val_accuracy: 0.7917 - val_auc: 0.7917 - val_loss: 0.5752 - val_precision: 0.8182 - val_recall: 0.7500 - learning_rate: 3.0000e-04
Epoch 3/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6842 - auc: 0.6983 - loss: 0.6470 - precision: 0.6677 - recall: 0.7192
Epoch 3: val_accuracy did not improve from 0.79167
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.6882 - auc: 0.7102 - loss: 0.6381 - precision: 0.6772 - recall: 0.7116 - val_accuracy: 0.7500 - val_auc: 0.8368 - val_loss: 0.5373 - val_precision: 0.7500 - val_recall: 0.7500 - learning_rate: 3.0000e-04
Epoch 4/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7351 - auc: 0.7500 - loss: 0.6000 - precision: 0.7579 - recall: 0.6548
Epoch 4: val_accuracy did not improve from 0.79167
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 3s/step - accuracy: 0.7163 - auc: 0.7463 - loss: 0.6073 - precision: 0.7360 - recall: 0.6508 - val

2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step - accuracy: 0.7937 - auc: 0.8921 - loss: 0.4376 - precision: 0.8232 - recall: 0.7707 - val_accuracy: 0.8333 - val_auc: 0.8819 - val_loss: 0.4472 - val_precision: 0.7500 - val_recall: 1.0000 - learning_rate: 3.0000e-04
Epoch 10/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6741 - auc: 0.7395 - loss: 0.6083 - precision: 0.6262 - recall: 0.7351
Epoch 10: val_accuracy did not improve from 0.83333
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 3s/step - accuracy: 0.6905 - auc: 0.7635 - loss: 0.5782 - precision: 0.6482 - recall: 0.7579 - val_accuracy: 0.8333 - val_auc: 0.8785 - val_loss: 0.4435 - val_precision: 0.7500 - val_recall: 1.0000 - learning_rate: 3.0000e-04
Epoch 11/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8013 - auc: 0.8632 - loss: 0.4417 - precision: 0.7687 - recall: 0.8893
Epoch 11: val_accuracy did not improve from 0.83333
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.8080 - auc: 0.8696 - loss: 0.4365 - precision: 0.7729 - recall: 0.8905 -


✅ Final Test Accuracy: 87.50%
📈 AUC: 0.8750 | 🎯 Precision: 0.8462 | 🔍 Recall: 0.9167
✅ Optimized MobileNetV2 training complete!
